<a href="https://colab.research.google.com/github/ckusmana21/DS2002-Data-Project-1/blob/main/Kusmana_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PART 1: REFLECTION



* Reflecting on this project, the part that I struggled with the most was, surprisingly, not building the code itself (in fact, this part was more streamlined thanks to internet resources and past class notes) -- but rather, understanding what the benchmarks are asking for. I think it requires that I understand the terms used. For instance, what does it mean to "ingest" a file? Does the script need to treat JSON and CSV files differently? How does one put "the option to convert any source to any target" - and what does that mean, in fact? I spent quite some time researching these questions and coming up with a simpler "checklist" for what the script needs to produce, as well as the correct deliverable and outputs. I found that having this "checklist" considerably helped me when I figured out how to get started with the code, and when I checked for whether the script was doing what I wanted it to do.
* I thought that conveting the differentfile formats (CSV to JSON and JSON to CSV) was easier than I thought, because pandas has simple methods (to_csv() and to_json(), both of which I used in part 2. A challenge that stood out to me, however, was writing codes that would allow users to specifcy the format that they wanted for this part. I researched stackoverflow (as indicated in the comments) to include all possible pathways for the data conversion. Nevertheless, another feature python had that greatly helped me is json_normalize() and pd.read_csv(). Once I found out how to assemble the codes together, the initial processing/ingesting, as well as the modifying steps, become simpler.
*In the future, I thought this utility would help me to integrate multiple data sources, especially when it comes to converting data from one format to another. An example that came to mind is converting CSV from a data dump to SQL for some cleanup, or to JSON for a publicy available API url upload. I did not do this for the project, but it would be interesting to see how similar codes would work when used to scrape websites and combining different dataset formats. It would help the data modification/transformation, too. This code, especially Part 3, would allow for quick conversion, in addition to letting us add and drop columns, all in the same pieline -- meaning, we don't have to write longer and more complicated scripts for just one output (that is , the modified file).
* Finally, another thing I appreciated about this project is that it worked with both smaller files (such as the one I am using to test the codes) and larger ones. Waiting time may differ, but the project seems scalable and versatile.



# PART 2: CODE

This is the first step of the process, which is creating the "skeleton" of the code. The second step is for processing the actual CSV and JSON files. For disclosure, I have indicated where I used internet resources or ChatGPT in the comment when appropriate, such as when fixing errors.

In [37]:
import pandas as pd
import json
import sqlite3
import requests

# PART 1a: This is for ingesting/processing the CSV file.
def ingest_csv(file_path):
    try:
        data = pd.read_csv(file_path)
        print(f"Data ingested from CSV: {len(data)} records and {len(data.columns)} columns")
        return data
    except Exception as e:
        print(f"Error ingesting CSV file: {e}")
        # NOTE: I consulted ChatGPT to create the line above. The prompt was, "how do I how do I write an error message if the CSV file doesn't exist or if I cannot open it?"
        return

# PART 1b: This is for ingesting/processing the JSON file.
def ingest_json(url):
    try:
        response = requests.get(url)
        data = response.json()
        df = pd.json_normalize(data)
        print(f"Data ingested from JSON: {len(df)} records and {len(df.columns)} columns")
        return df
    except Exception as e:
        print(f"Error fetching or parsing JSON: {e}")
        return None

# PART 2A: This is for converting the data.
# NOTE: I consulted "https://stackoverflow.com/questions/43876246/read-and-process-a-text-file-and-save-to-csv" to find out how to correctly save the files, where I found out about "convert_to_csv".
def convert_to_csv(df, output_path):
    df.to_csv(output_path, index=False)
    print(f"Data saved as CSV at {output_path}")

def convert_to_json(df, output_path):
    df.to_json(output_path, orient='records')
    print(f"Data saved as JSON at {output_path}")

# PART 2B: This is for converting the data, BUT based on user's choice.
def convert_data(data, output_format, output_path, db_path=None, table_name=None):
    if output_format == 'csv':
        # Convert to CSV
        convert_to_csv(data, output_path)
    elif output_format == 'json':
        # Convert to JSON
        convert_to_json(data, output_path)
    elif output_format == 'sql':
        if db_path and table_name:
            # Convert to SQL database
            save_to_sql(data, db_path, table_name)
        else:
            print("Please provide both db_path and table_name for SQL conversion.")
    else:
        print("Unsupported output format. Please choose 'csv', 'json', or 'sql'.")


# PART 3: This is for modyfing the columns - such as, adding or dropping columns.
def modify_columns(df, columns_to_drop=None, new_columns=None):
    if columns_to_drop:
        df = df.drop(columns=columns_to_drop)
    if new_columns:
        for col, value in new_columns.items():
            df[col] = value
    print(f"Modified data: {len(df)} records and {len(df.columns)} columns")
    return df

# PART 4: This is for savaing the data files (CSV or JSON) locally.
def save_to_local(df, output_path, file_format):
    if file_format == 'csv':
        convert_to_csv(df, output_path)
    elif file_format == 'json':
        convert_to_json(df, output_path)
    else:
        print("Unsupported file format. Make sure to use CSV or JSON")

# PART 5 comes in the next part. It is also combined in PART 1a and PART 1b.

This is for testing the codes using two datasets: one a CSV (from another class) containing information of car sales in the U.S, and another a JSON file (its API url is publicly available) containing information of colleges/universities in the U.S.


*   I have chosen to save the files locally. However, I included a potential way to save it on a SQL database at the end as a comment.
*   I tested the modified files at the end (head of each)



In [38]:
# PART 5 (summary)
if __name__ == "__main__":
    csv_data = ingest_csv('/content/USA_cars_datasets.csv')
    json_data = ingest_json('http://universities.hipolabs.com/search?country=United+States')

    # This is for modifying the data. I am dropping the column "country" and adding "price" for the CSV, and adding the column "Global Rnaking" in the JSON
    # You can customize which columns to drop or add
    csv_data2 = modify_columns(csv_data, columns_to_drop=['country'], new_columns={'price': 'placeholder value'}) # here, the placeholder values are the prices of the cars.
    json_data2 = modify_columns(json_data, new_columns={'Global Ranking': 'another placeholder value'}) # here, the placeholder values are the ranking of the schools.

    # This is for saveing the newly modified files locally. Frist for the CSV, and second for the JSON.
    save_to_local(csv_data2, 'processed_cars_data.csv', 'csv')
    save_to_local(json_data2, 'processed_universities_data.json', 'json')

    # This is for saving the data to SQL instead of locally. However, I wasn't able to run this correctly, so I'm including this as a comment:
    ## save_to_sql(json_data2, 'data.db', 'json_table')


Data ingested from CSV: 2499 records and 13 columns
Data ingested from JSON: 2335 records and 6 columns
Modified data: 2499 records and 12 columns
Modified data: 2335 records and 7 columns
Data saved as CSV at processed_cars_data.csv
Data saved as JSON at processed_universities_data.json


This is just to view the first few rows of both files to see if they have been modified correctly, confriming that the the values in "Data ingested from __" and "Modified data" are correct.

In [28]:
print(csv_data2.head())

   Unnamed: 0              price      brand    model  year   title_status  \
0           0  placeholder value     toyota  cruiser  2008  clean vehicle   
1           1  placeholder value       ford       se  2011  clean vehicle   
2           2  placeholder value      dodge      mpv  2018  clean vehicle   
3           3  placeholder value       ford     door  2014  clean vehicle   
4           4  placeholder value  chevrolet     1500  2018  clean vehicle   

   mileage   color                  vin        lot       state      condition  
0   274117   black    jtezu11f88k007763  159348797  new jersey   10 days left  
1   190552  silver    2fmdk3gc4bbb02217  166951262   tennessee    6 days left  
2    39590  silver    3c4pdcgg5jt346413  167655728     georgia    2 days left  
3    64146    blue    1ftfw1et4efc23745  167753855    virginia  22 hours left  
4     6654     red    3gcpcrec2jg473991  167763266     florida  22 hours left  


In [29]:
print(json_data2.head())

                                    name alpha_two_code  \
0                    Marywood University             US   
1                  Lindenwood University             US   
2                    Sullivan University             US   
3  Florida State College at Jacksonville             US   
4                      Xavier University             US   

                      web_pages state-province           domains  \
0     [http://www.marywood.edu]           None    [marywood.edu]   
1  [http://www.lindenwood.edu/]           None  [lindenwood.edu]   
2       [https://sullivan.edu/]           None    [sullivan.edu]   
3       [https://www.fscj.edu/]           None        [fscj.edu]   
4     [https://www.xavier.edu/]           None      [xavier.edu]   

         country             Global Ranking  
0  United States  another placeholder value  
1  United States  another placeholder value  
2  United States  another placeholder value  
3  United States  another placeholder value  
4  Uni

PART 2b: This is specifically for converting the files to the desired output.


In [39]:
# PART 2b: This is specifically for converting the files to the desired output.
if __name__ == "__main__":
    # This will allow user to select input. NOTE: I found out how to use the "input" function from "https://stackoverflow.com/questions/3345202/getting-user-input".
    input_type = input("Enter the input type (csv or json): ").lower()
    output_type = input("Enter the chosen output type (csv or json or sql): ").lower()

    # This is for processing/ingesting the data
    if input_type == 'csv':
        data = ingest_csv('/content/USA_cars_datasets.csv')
    elif input_type == 'json':
        data = ingest_json('http://universities.hipolabs.com/search?country=United+States')
    else:
        print("Unsupported input type. Please choose 'csv' or 'json'.")
        data = None

    # This is for converting data type, based on user's chocies.
    if data is not None:
        if output_type == 'csv':
            convert_data(data, 'csv', 'output.csv')
        elif output_type == 'json':
            convert_data(data, 'json', 'output.json')
        elif output_type == 'sql':
            # NOTE: for the SQL converstion, user have to put in the database path and name of the table
            db_path = input("Enter the path for the database (e.g., 'data.db'): ")
            table_name = input("Enter the table name for the SQL database: ")
            convert_data(data, 'sql', output_path=None, db_path=db_path, table_name=table_name)
        else:
            print("Unsupported output type. Please choose 'csv', 'json', or 'sql'.")


Enter the input type (csv or json): CSV
Enter the chosen output type (csv or json or sql): JSON
Data ingested from CSV: 2499 records and 13 columns
Data saved as JSON at output.json
